In [ ]:
! pip -q install transformers datasets

In [1]:
! rm -rf memes-dataset.csv
! wget https://github.com/mrsndmn/memes-dataset/raw/main/memes-dataset.csv

--2023-01-06 02:14:16--  https://github.com/mrsndmn/memes-dataset/raw/main/memes-dataset.csv
Распознаётся github.com (github.com)… 140.82.121.4
Подключение к github.com (github.com)|140.82.121.4|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/mrsndmn/memes-dataset/main/memes-dataset.csv [переход]
--2023-01-06 02:14:17--  https://raw.githubusercontent.com/mrsndmn/memes-dataset/main/memes-dataset.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 4608389 (4,4M) [application/octet-stream]
Сохранение в: «memes-dataset.csv»

memes-dataset.csv   100%[===================>]   4,39M  1,77MB/s    за 2,5s    

2023-01-06 02:14:20 (1,77 MB/s) - «memes-dataset.csv» сохранён [4608389/

In [1]:
from datasets import load_dataset

In [2]:
import torch
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('mps')
print(device)

mps


# Датасет

Описание датасета можно прочитать в нулевой домашке (см ссылку в пине чатика)

In [3]:
!head memes-dataset.csv

template,mem_href,mem_description
110133729/spiderman-pointing-at-spiderman,/i/5whzkb,"
IMAGE DESCRIPTION:
IF YOU HAVE A TWIN YOUR LUCKY "
110133729/spiderman-pointing-at-spiderman,/i/5w2wcr,"
IMAGE DESCRIPTION:
ME; DAD; ME AND MY DAD WHEN MOM SCREAMS WHO ATE THE LAST COOKIE "
110133729/spiderman-pointing-at-spiderman,/i/5uodc5,"
IMAGE DESCRIPTION:
ME; A KID WEARING THE SAME SHIRT I'M WEARING "


In [4]:
dataset = load_dataset('csv', data_files={"train": "memes-dataset.csv"})

Using custom data configuration default-c344730d982eba42
Found cached dataset csv (/Users/vsglamazdin/.cache/huggingface/datasets/csv/default-c344730d982eba42/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset['train']

Dataset({
    features: ['template', 'mem_href', 'mem_description'],
    num_rows: 39315
})

**Задание 1. [0.25 балла]** Переименуйте колонки датасета.  Название колонок должно соответствовать названиями колонок в других датасетах, с которыми работают модели из Transformers: должны быть колонки `text` и `label`

Используйте `dataset.rename_columns`.

**Задание 2. [0.25 балла]** Отфильтруйте из датасета строчки, в которых нет текста.

In [6]:
dataset = dataset.rename_columns({'mem_description': 'text', 'template': 'label'}) # rename
dataset = dataset.filter(lambda x: x['text'] != None) # filter

dataset

Loading cached processed dataset at /Users/vsglamazdin/.cache/huggingface/datasets/csv/default-c344730d982eba42/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-f531fb24eaab5a07.arrow


DatasetDict({
    train: Dataset({
        features: ['label', 'mem_href', 'text'],
        num_rows: 38783
    })
})

In [7]:
dataset['train'][0]

{'label': '110133729/spiderman-pointing-at-spiderman',
 'mem_href': '/i/5whzkb',
 'text': '\nIMAGE DESCRIPTION:\nIF YOU HAVE A TWIN YOUR LUCKY '}

**Задание 3. [0.25 балла]** Заметили, что каждый текст наинается с одного и того же префикса? Давайте обрежим этот префикс. Испоьлзуйте `dataset.map`

In [8]:
def cut_prefix(example):
    if example['text'].find('\nIMAGE DESCRIPTION:\n') == 0:
        example['text'] = example['text'].replace('\nIMAGE DESCRIPTION:\n', '', 1)
    return example


dataset = dataset.map(cut_prefix) # map

Loading cached processed dataset at /Users/vsglamazdin/.cache/huggingface/datasets/csv/default-c344730d982eba42/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-f31864b5d39aa540.arrow


In [9]:
dataset['train'][0]

{'label': '110133729/spiderman-pointing-at-spiderman',
 'mem_href': '/i/5whzkb',
 'text': 'IF YOU HAVE A TWIN YOUR LUCKY '}

**Задание 4. [0.25 балла]** Закодируйте метки датасета с помощью LabelEncoder и `dataset.map`.

Для того, чтобы получить список уникальных лейблов можно, например, превратить датасет сначала в датафрейм: `dataset['train'].to_pandas()`. А как получить из колонки датафрейма уникальные значения мы знаем.

In [10]:
from sklearn.preprocessing import LabelEncoder

In [15]:
templates = dataset['train'].to_pandas()['label'].unique() # get uniq templates
templates[:3]

array([0, 1, 2])

In [16]:
le = LabelEncoder().fit(templates) # label encoder + fit
le

LabelEncoder()

In [17]:
import numpy as np

def le_transform(example):
    example['label'] = le.transform(np.array([example['label']]))[0]
    return example
dataset = dataset.map(le_transform) # map dataset


  0%|          | 0/38783 [00:00<?, ?ex/s]

# Токенизация

Нужно токенизировать датасет. В качестве модели (в том числе чеккпоинт для токенайзера) используйте https://huggingface.co/distilbert-base-uncased

Важно посмотреть на результат токенизации: не будет ли там ничего лишнего?

**Задание 5. [0.25 балла]** Загрузите токенизатор и токенизируйте датасет

In [18]:
from transformers import AutoTokenizer

MODEL_CHECKPOINT = 'distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, use_fast=True) # load tokenizer

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/39 [00:00<?, ?ba/s]

Разобъем датасет на трейн/тест

Эту ячейку менять нельзя

In [19]:
dataset = dataset['train'].train_test_split(test_size=0.05, seed=42)

**Задание 6. [0.25 балла]** Смотрим, как работает токенизатор

Проведите токенизацию первого предложения в тексте. Напечатайте исходное предложение, потом выведете список токенов (это "массив" чисел), потом сделайте обратное преобразование списка токенов с помощью `tokenizer.decode`

In [20]:
text = dataset['train'][0]['text']  # get text
print(text)

inputs_ids = tokenizer(text)['input_ids'] # get tokenized text
print(inputs_ids)

detokenized = tokenizer.decode(inputs_ids) # decode tokenized text
print(detokenized)

RICK TELLING MORTY ABOUT ALL THE OTHER ADVENTURES THAT THE NEED TO GO ON 
[101, 6174, 4129, 22294, 2100, 2055, 2035, 1996, 2060, 7357, 2008, 1996, 2342, 2000, 2175, 2006, 102]
[CLS] rick telling morty about all the other adventures that the need to go on [SEP]


In [21]:
tokenized_datasets['train'][0]['label']

0

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'mem_href', 'text', 'input_ids', 'attention_mask'],
        num_rows: 38783
    })
})

Загрузите модель из того же чекпонта, что и токенайзер

# Модель

In [23]:
from transformers import AutoModelForSequenceClassification

num_labels = len(templates) #

model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=num_labels).to(device)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

# Обучение

**Задание 7. [2 балла]** Обучите модель так, чтобы accuracy на валидационном датасете было не меньше `0.4`

Обратите внимание на `batch_size`, `learning_rate`, другие гиперпараметры обучения. Можно дообработать датасет, если вы считаете это необходимым. Возможно, трейн несбалансированный, возможно, он слишком большой

In [24]:
from transformers import TrainingArguments

In [25]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


/var/folders/g9/rpvrft0n46j_zy6tctl1vmgw0000gn/T/ipykernel_4051/2014915044.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [26]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16 ,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    seed=42,
    learning_rate=1e-4,
    save_steps=1000
)

In [27]:
tokenized_datasets = tokenized_datasets['train'].train_test_split(test_size=0.05, seed=42)

In [28]:
# если возникает ошибка с CUDA можно попробовать установить эту переменную изи воспроизвести ошибку в колабе в среде без GPU
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: mem_href, text. If mem_href, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/vsglamazdin/opt/anaconda3/envs/notebookkernel/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 36843
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6909
  Number of trainable parameters = 66986539
You're using a DistilBertTokenizerFast tokenizer. Please note tha

Epoch,Training Loss,Validation Loss


In [27]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: mem_href, text. If mem_href, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 36843
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6909


Epoch,Training Loss,Validation Loss,Accuracy
1,1.701000,1.522773,0.593299
2,1.190500,1.421965,0.623711
3,0.744000,1.497390,0.633505


Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-2000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-2000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: mem_href, text. If mem_href, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
**

TrainOutput(global_step=6909, training_loss=1.2971365394873873, metrics={'train_runtime': 5223.2967, 'train_samples_per_second': 21.161, 'train_steps_per_second': 1.323, 'total_flos': 1.4652194623128576e+16, 'train_loss': 1.2971365394873873, 'epoch': 3.0})

In [28]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: mem_href, text. If mem_href, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1940
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.6335051546391752,
 'eval_loss': 1.4973896741867065,
 'eval_runtime': 34.9648,
 'eval_samples_per_second': 55.484,
 'eval_steps_per_second': 3.489}

**Задание 8. [0.25 балла]**  Какой `data_collator` используется в процессе обучения? Зачем он нужен? Покажите, как он работает?


**Ответ:** (ожидается текстовый ответ на первые 2 вопроса и ячейка, которая демонстрирует работу `data_collator`'а)

In [29]:
data_collator = trainer.data_collator
print(f'Используется следующий data_collator:\n{data_collator}')

Используется следующий data_collator:
DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')


data_collator образует батчи из тензоров, используя в качестве входных данных список элементов датасета. Для этого data_collator может осуществлять некоторую предобработку. Например, наш data_collator добавляет padding, так как в общем случае входные последовательности токенов могут быть разной длины (у меня в токенизаторе делаются одинаковой длины).

In [30]:
batch = {'input_ids': [tokenized_datasets['train'][0]['input_ids'], tokenized_datasets['train'][1]['input_ids']], 
         'label': [42, 31]}
data_collator(batch)

{'input_ids': tensor([[ 101, 6174, 4129,  ...,    0,    0,    0],
        [ 101, 2111, 2006,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([42, 31])}

In [31]:
batch = {'input_ids': [[0,1,2], [0,1,2,3,4,5]], 'label': [1, 13]}
data_collator(batch)

{'input_ids': tensor([[0, 1, 2, 0, 0, 0],
        [0, 1, 2, 3, 4, 5]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1]]), 'labels': tensor([ 1, 13])}

# Тестирование

**Задание 9. [0.25 балла]**  Создайте text-classification pipeline, который будет использовать вашу обученную модель и токеннайзер, протестируйте вашу модель

In [32]:
from transformers import pipeline
classification_pipeline = pipeline('text-classification', model=model.cpu(), tokenizer=tokenizer)

In [33]:
classification_pipeline("i bet hes thinking about me")

[{'label': 'LABEL_30', 'score': 0.9993007183074951}]

In [34]:
le.inverse_transform([30])


array(['I-Bet-Hes-Thinking-About-Other-Women'], dtype=object)

In [35]:
dataset['train'][0]

{'label': 42,
 'mem_href': '/i/5mz58c',
 'text': 'RICK TELLING MORTY ABOUT ALL THE OTHER ADVENTURES THAT THE NEED TO GO ON '}

In [36]:
classification_pipeline('ADVENTURES THAT THE NEED TO GO ON')

[{'label': 'LABEL_42', 'score': 0.8658950924873352}]

In [37]:
le.inverse_transform([42])

array(['X-X-Everywhere'], dtype=object)

In [38]:
dataset['train'][100]

{'label': 29,
 'mem_href': '/i/4tyi8d',
 'text': 'MY FRIEND HAS A PICTURE OF MY WIFE AND HIM KISSING '}

In [39]:
classification_pipeline('friend photo with my wife')

[{'label': 'LABEL_29', 'score': 0.7758901715278625}]

In [40]:
le.inverse_transform([29])

array(['Hide-the-Pain-Harold'], dtype=object)

In [77]:
dataset['test'][13]

{'label': 41,
 'mem_href': '/i/5pu8xo',
 'text': 'Video games cause violence! Me, just trying to raise my Pokemon '}

In [78]:
classification_pipeline("violently catching pokemon"), classification_pipeline("Video games cause violence!")

([{'label': 'LABEL_38', 'score': 0.818120539188385}],
 [{'label': 'LABEL_41', 'score': 0.9934084415435791}])

In [79]:
le.inverse_transform([38]), le.inverse_transform([41])

(array(['This-Is-Fine'], dtype=object),
 array(['Woman-Yelling-At-Cat'], dtype=object))

In [80]:
dataset['test'][43]

{'label': 20,
 'mem_href': '/i/52edpy',
 'text': 'me thinking i am cool and standing on my bike; me starting to fall; me realising i am not cool '}

In [81]:
classification_pipeline('cool bike')

[{'label': 'LABEL_20', 'score': 0.9791275858879089}]

In [82]:
le.inverse_transform([20])

array(['Bike-Fall'], dtype=object)

In [83]:
dataset['test'][107]

{'label': 32,
 'mem_href': '/i/3rbwzc',
 'text': 'WHEN SHE SAYS YES TO GO ON A DATE '}

In [84]:
classification_pipeline('WHEN SHE SAYS YES TO GO ON A DATE ')

[{'label': 'LABEL_36', 'score': 0.18404856324195862}]

In [85]:
le.inverse_transform([32]), le.inverse_transform([36])

(array(['Leonardo-Dicaprio-Cheers'], dtype=object),
 array(['Success-Kid'], dtype=object))